In [ ]:
import sqlite3
import pandas as pd
import os
from datetime import datetime

def connect_to_database(db_path):
    """
    Connect to the specified database
    """
    try:
        if not os.path.exists(db_path):
            print(f"❌ Database file not found: {db_path}")
            return None
        
        conn = sqlite3.connect(db_path)
        print(f"✅ Successfully connected to database: {db_path}")
        return conn
    except sqlite3.Error as e:
        print(f"❌ Error connecting to database: {e}")
        return None

def analyze_database_structure(conn):
    """
    Analyze the database to find tables with date and buy columns
    """
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        
        print(f"📊 Found {len(tables)} table(s) in the database")
        
        suitable_tables = {}
        
        for table in tables:
            table_name = table[0]
            print(f"\n🔸 Analyzing table: {table_name}")
            
            # Get column info
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = cursor.fetchall()
            column_names = [col[1].lower() for col in columns]
            original_columns = [col[1] for col in columns]
            
            # Look for date columns
            date_columns = [col for col in original_columns if any(keyword in col.lower() 
                           for keyword in ['date', 'time', 'timestamp', 'day'])]
            
            # Look for buy columns
            buy_columns = [col for col in original_columns if any(keyword in col.lower() 
                          for keyword in ['buy', 'purchase', 'bid'])]
            
            # Look for instrument columns
            instrument_columns = [col for col in original_columns if any(keyword in col.lower() 
                                 for keyword in ['symbol', 'instrument', 'ticker', 'asset', 'code'])]
            
            print(f"   Date columns found: {date_columns}")
            print(f"   Buy columns found: {buy_columns}")
            print(f"   Instrument columns found: {instrument_columns}")
            
            if date_columns and buy_columns and instrument_columns:
                # Get row count
                cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
                row_count = cursor.fetchone()[0]
                
                suitable_tables[table_name] = {
                    'date_columns': date_columns,
                    'buy_columns': buy_columns,
                    'instrument_columns': instrument_columns,
                    'row_count': row_count,
                    'all_columns': original_columns
                }
                print(f"   ✅ Suitable table with {row_count:,} rows")
            else:
                print(f"   ⚠️  Missing required columns")
        
        return suitable_tables
        
    except sqlite3.Error as e:
        print(f"❌ Error analyzing database: {e}")
        return None

def get_instruments_and_create_csvs(conn, suitable_tables):
    """
    Extract data for each instrument and create CSV files
    """
    if not suitable_tables:
        print("❌ No suitable tables found")
        return
    
    # Create output directory
    output_dir = f"instrument_csvs_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    os.makedirs(output_dir, exist_ok=True)
    print(f"📁 Created output directory: {output_dir}")
    
    total_files_created = 0
    
    for table_name, table_info in suitable_tables.items():
        print(f"\n📊 Processing table: {table_name}")
        
        # Use first available columns
        date_col = table_info['date_columns'][0]
        buy_col = table_info['buy_columns'][0]
        instrument_col = table_info['instrument_columns'][0]
        
        print(f"   Using columns: {instrument_col} (instrument), {date_col} (date), {buy_col} (buy)")

        try:
            # Get unique instruments
            cursor = conn.cursor()
            cursor.execute(f"SELECT DISTINCT {instrument_col} FROM {table_name} WHERE {instrument_col} IS NOT NULL ORDER BY {instrument_col};")
            instruments = cursor.fetchall()
            
            print(f"   Found {len(instruments)} unique instruments")
            
            for instrument_tuple in instruments:
                instrument = instrument_tuple[0]
                if instrument is None:
                    continue
                
                try:
                    # Query data for this instrument
                    query = f"""
                    SELECT {date_col} as date, {buy_col} as buy
                    FROM {table_name}
                    WHERE {instrument_col} = ?
                    AND {date_col} IS NOT NULL
                    AND {buy_col} IS NOT NULL
                    ORDER BY {date_col}
                    LIMIT 2
                    """
                    
                    df = pd.read_sql_query(query, conn, params=[instrument])

                    print(instrument)
                    print(df)
                    print('_'*20)
                    
                    
                        
                except Exception as e:
                    print(f"   ❌ Error processing {instrument}: {e}")
            
        except sqlite3.Error as e:
            print(f"❌ Error processing table {table_name}: {e}")
    
    print(f"\n🎯 Summary:")
    print(f"   Total CSV files created: {total_files_created}")
    print(f"   Output directory: {output_dir}")
    
    # List some of the created files
    if total_files_created > 0:
        files = os.listdir(output_dir)
        print(f"   Sample files created:")
        for i, file in enumerate(files[:10]):  # Show first 10 files
            file_path = os.path.join(output_dir, file)
            file_size = os.path.getsize(file_path)
            print(f"     • {file} ({file_size} bytes)")
        
        if len(files) > 10:
            print(f"     ... and {len(files) - 10} more files")


# Main execution
db_path = "aia_big.db"

# Connect to database
conn = connect_to_database(db_path)

if conn:
    print("\n" + "="*60)
    print("🔍 ANALYZING DATABASE STRUCTURE")
    print("="*60)
    
    # Analyze database structure
    suitable_tables = analyze_database_structure(conn)
    
    if suitable_tables:
        print("\n" + "="*60)
        print("📥 CREATING CSV FILES FOR EACH INSTRUMENT")
        print("="*60)
        
        # Create CSV files for each instrument
        get_instruments_and_create_csvs(conn, suitable_tables)
        
    
    conn.close()
    print(f"\n✅ Database connection closed")
else:
    print("❌ Could not connect to database")